# SWARM Intelligence Optimization

**Use Case: Autonomous Research & Discovery**

This notebook demonstrates QBitaLabs' bio-inspired SWARM coordination patterns for complex optimization problems.

## SWARM Patterns

| Pattern | Inspiration | Best For |
|---------|-------------|----------|
| **Protein Swarm** | Protein folding | Molecular design |
| **Stigmergy** | Ant pheromones | Path finding |
| **Ant Colony** | Foraging ants | Combinatorial optimization |
| **Particle Swarm** | Bird flocking | Continuous optimization |
| **Hierarchical** | Bee colonies | Multi-scale problems |

In [ ]:
import asyncio
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# QBitaLabs SWARM imports
from qbitalabs.swarm import SwarmFabric
from qbitalabs.swarm.patterns import (
    ProteinSwarmPattern,
    StigmergyPattern,
    AntColonyOptimization,
    ParticleSwarmOptimization,
    HierarchicalPattern
)
from qbitalabs.swarm.agents import MolecularAgent, HypothesisAgent

print("QBitaLabs SWARM Module loaded!")
print(f"Session: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 1. Protein Swarm: Molecular Design

Agents coordinate like proteins - binding, folding, and forming complexes.

In [ ]:
# Configure protein swarm pattern
protein_swarm = ProteinSwarmPattern(
    binding_affinity_threshold=0.7,
    folding_temperature=0.5,
    complex_size_limit=5
)

print("Protein Swarm Pattern Configuration:")
print(f"  Binding threshold: {protein_swarm.binding_affinity_threshold}")
print(f"  Folding temperature: {protein_swarm.folding_temperature}")
print(f"  Max complex size: {protein_swarm.complex_size_limit}")

In [ ]:
# Simulated protein swarm optimization
iterations = 50
binding_events = []
best_fitness = []

np.random.seed(42)
fitness = 0.3

for i in range(iterations):
    # Simulate binding probability
    binding_prob = 0.3 + 0.5 * (i / iterations)  # Increases over time
    binding_events.append(int(np.random.random() < binding_prob))
    
    # Fitness improves with binding events
    if binding_events[-1]:
        fitness = min(1.0, fitness + np.random.uniform(0.02, 0.08))
    best_fitness.append(fitness)

print(f"\nProtein Swarm Optimization Results:")
print(f"  Total binding events: {sum(binding_events)}")
print(f"  Initial fitness: {best_fitness[0]:.3f}")
print(f"  Final fitness: {best_fitness[-1]:.3f}")
print(f"  Improvement: {(best_fitness[-1] - best_fitness[0]) / best_fitness[0]:.0%}")

In [ ]:
# Visualize protein swarm optimization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Fitness trajectory
ax1.plot(best_fitness, 'b-', linewidth=2)
ax1.scatter([i for i, b in enumerate(binding_events) if b], 
           [best_fitness[i] for i, b in enumerate(binding_events) if b],
           c='red', s=50, zorder=5, label='Binding event')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Best Fitness')
ax1.set_title('Protein Swarm: Fitness Trajectory')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Binding event rate
window = 10
binding_rate = [sum(binding_events[max(0,i-window):i+1])/min(window, i+1) 
               for i in range(iterations)]
ax2.plot(binding_rate, 'g-', linewidth=2)
ax2.set_xlabel('Iteration')
ax2.set_ylabel('Binding Rate (rolling window)')
ax2.set_title('Agent Cooperation Rate')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Ant Colony Optimization: Drug Combination Search

In [ ]:
# Configure ACO for drug combination optimization
aco = AntColonyOptimization(
    num_ants=20,
    evaporation_rate=0.1,
    pheromone_constant=1.0,
    alpha=1.0,  # Pheromone importance
    beta=2.0    # Heuristic importance
)

# Drug combination problem
drugs = ["Drug A", "Drug B", "Drug C", "Drug D", "Drug E", "Drug F"]
num_drugs = len(drugs)

# Simulated efficacy matrix (synergy scores)
np.random.seed(42)
synergy_matrix = np.random.uniform(0.5, 1.5, (num_drugs, num_drugs))
np.fill_diagonal(synergy_matrix, 1.0)  # Self-combination = neutral

print("Ant Colony Optimization for Drug Combinations")
print(f"  Number of drugs: {num_drugs}")
print(f"  Number of ants: {aco.num_ants}")
print(f"  Possible 2-drug combinations: {num_drugs * (num_drugs-1) // 2}")

In [ ]:
# Run ACO simulation
iterations = 30
pheromone_matrix = np.ones((num_drugs, num_drugs))
best_combinations = []
best_scores = []

for iteration in range(iterations):
    iteration_best = 0
    iteration_best_combo = None
    
    # Each ant finds a path
    for ant in range(aco.num_ants):
        # Select two drugs based on pheromone and heuristic
        probabilities = pheromone_matrix ** aco.alpha * synergy_matrix ** aco.beta
        # Normalize
        row_sums = probabilities.sum(axis=1, keepdims=True)
        probabilities = probabilities / row_sums
        
        # Random selection based on probabilities
        drug1 = np.random.choice(num_drugs)
        drug2 = np.random.choice(num_drugs, p=probabilities[drug1]/probabilities[drug1].sum())
        
        score = synergy_matrix[drug1, drug2]
        if score > iteration_best:
            iteration_best = score
            iteration_best_combo = (drug1, drug2)
    
    # Update pheromones
    pheromone_matrix *= (1 - aco.evaporation_rate)  # Evaporation
    if iteration_best_combo:
        i, j = iteration_best_combo
        pheromone_matrix[i, j] += aco.pheromone_constant * iteration_best
        pheromone_matrix[j, i] += aco.pheromone_constant * iteration_best
    
    best_combinations.append(iteration_best_combo)
    best_scores.append(iteration_best)

# Find overall best
overall_best_idx = np.argmax(best_scores)
overall_best_combo = best_combinations[overall_best_idx]

print(f"\nACO Results:")
print(f"  Best combination: {drugs[overall_best_combo[0]]} + {drugs[overall_best_combo[1]]}")
print(f"  Synergy score: {best_scores[overall_best_idx]:.3f}")
print(f"  Found at iteration: {overall_best_idx + 1}")

In [ ]:
# Visualize ACO results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Best score over iterations
ax1.plot(best_scores, 'b-o', linewidth=2, markersize=4)
ax1.axhline(y=max(best_scores), color='r', linestyle='--', label=f'Best: {max(best_scores):.3f}')
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Best Synergy Score')
ax1.set_title('ACO Convergence')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Final pheromone matrix
im = ax2.imshow(pheromone_matrix, cmap='YlOrRd')
ax2.set_xticks(range(num_drugs))
ax2.set_yticks(range(num_drugs))
ax2.set_xticklabels([d.split()[1] for d in drugs])
ax2.set_yticklabels([d.split()[1] for d in drugs])
ax2.set_xlabel('Drug')
ax2.set_ylabel('Drug')
ax2.set_title('Final Pheromone Levels')
plt.colorbar(im, ax=ax2, label='Pheromone')

plt.tight_layout()
plt.show()

## 3. Particle Swarm: Parameter Optimization

In [ ]:
# Configure PSO for continuous optimization
pso = ParticleSwarmOptimization(
    num_particles=30,
    inertia_weight=0.7,
    cognitive_weight=1.5,
    social_weight=1.5
)

# Example: Optimize drug dosing parameters
# Objective: Find optimal dose and timing to maximize efficacy - toxicity
def efficacy_toxicity_function(dose, timing):
    """Simulated drug response function."""
    efficacy = 2 * np.exp(-((dose - 50)**2 / 500 + (timing - 12)**2 / 50))
    toxicity = 0.01 * dose**1.5 + 0.001 * (24 - timing)**2
    return efficacy - toxicity

print("Particle Swarm Optimization for Drug Dosing")
print(f"  Number of particles: {pso.num_particles}")
print(f"  Parameters: dose (0-100 mg), timing (0-24 h)")

In [ ]:
# Run PSO
iterations = 50
np.random.seed(42)

# Initialize particles
positions = np.random.rand(pso.num_particles, 2) * [100, 24]  # dose, timing
velocities = np.random.randn(pso.num_particles, 2) * [5, 1]
personal_best_pos = positions.copy()
personal_best_val = np.array([efficacy_toxicity_function(p[0], p[1]) for p in positions])
global_best_idx = personal_best_val.argmax()
global_best_pos = personal_best_pos[global_best_idx].copy()
global_best_val = personal_best_val[global_best_idx]

history = {'global_best': [], 'positions': []}

for iteration in range(iterations):
    # Store history
    history['global_best'].append(global_best_val)
    history['positions'].append(positions.copy())
    
    # Update velocities and positions
    r1, r2 = np.random.rand(2)
    cognitive = pso.cognitive_weight * r1 * (personal_best_pos - positions)
    social = pso.social_weight * r2 * (global_best_pos - positions)
    velocities = pso.inertia_weight * velocities + cognitive + social
    positions = positions + velocities
    
    # Clip to bounds
    positions[:, 0] = np.clip(positions[:, 0], 0, 100)
    positions[:, 1] = np.clip(positions[:, 1], 0, 24)
    
    # Evaluate
    current_val = np.array([efficacy_toxicity_function(p[0], p[1]) for p in positions])
    
    # Update personal bests
    improved = current_val > personal_best_val
    personal_best_pos[improved] = positions[improved]
    personal_best_val[improved] = current_val[improved]
    
    # Update global best
    if current_val.max() > global_best_val:
        global_best_idx = current_val.argmax()
        global_best_pos = positions[global_best_idx].copy()
        global_best_val = current_val[global_best_idx]

print(f"\nPSO Results:")
print(f"  Optimal dose: {global_best_pos[0]:.1f} mg")
print(f"  Optimal timing: {global_best_pos[1]:.1f} hours")
print(f"  Objective value: {global_best_val:.4f}")

In [ ]:
# Visualize PSO
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Convergence
ax1 = axes[0]
ax1.plot(history['global_best'], 'b-', linewidth=2)
ax1.set_xlabel('Iteration')
ax1.set_ylabel('Best Objective Value')
ax1.set_title('PSO Convergence')
ax1.grid(True, alpha=0.3)

# Objective landscape
ax2 = axes[1]
dose_range = np.linspace(0, 100, 50)
timing_range = np.linspace(0, 24, 50)
D, T = np.meshgrid(dose_range, timing_range)
Z = efficacy_toxicity_function(D, T)
contour = ax2.contourf(D, T, Z, levels=20, cmap='RdYlGn')
ax2.scatter(global_best_pos[0], global_best_pos[1], c='red', s=200, marker='*', 
           edgecolors='black', linewidths=2, label='Optimal', zorder=5)
ax2.set_xlabel('Dose (mg)')
ax2.set_ylabel('Timing (hours)')
ax2.set_title('Efficacy-Toxicity Landscape')
ax2.legend()
plt.colorbar(contour, ax=ax2, label='Objective')

# Particle trajectories (first 5 particles)
ax3 = axes[2]
for p in range(min(5, pso.num_particles)):
    traj = np.array([history['positions'][i][p] for i in range(iterations)])
    ax3.plot(traj[:, 0], traj[:, 1], 'o-', alpha=0.5, markersize=2)
ax3.scatter(global_best_pos[0], global_best_pos[1], c='red', s=200, marker='*', 
           edgecolors='black', linewidths=2, zorder=5)
ax3.set_xlabel('Dose (mg)')
ax3.set_ylabel('Timing (hours)')
ax3.set_title('Particle Trajectories (5 particles)')
ax3.set_xlim(0, 100)
ax3.set_ylim(0, 24)

plt.tight_layout()
plt.show()

## 4. Hierarchical Pattern: Multi-Scale Research

Strategic → Planning → Execution agents work at different scales.

In [ ]:
# Configure hierarchical pattern
hierarchical = HierarchicalPattern()

# Example: Drug discovery hierarchy
hierarchy = {
    "Strategic (CEO-level)": {
        "agents": 1,
        "decisions": ["Target selection", "Resource allocation", "Risk assessment"],
        "time_horizon": "Years"
    },
    "Planning (Director-level)": {
        "agents": 3,
        "decisions": ["Compound prioritization", "Assay design", "Timeline planning"],
        "time_horizon": "Months"
    },
    "Execution (Scientist-level)": {
        "agents": 10,
        "decisions": ["Molecule synthesis", "Data analysis", "Experiment execution"],
        "time_horizon": "Days"
    }
}

print("\n" + "="*60)
print("HIERARCHICAL SWARM PATTERN")
print("="*60)
for level, config in hierarchy.items():
    print(f"\n{level}:")
    print(f"  Agents: {config['agents']}")
    print(f"  Time horizon: {config['time_horizon']}")
    print(f"  Decisions: {', '.join(config['decisions'])}")

## Customer Applications

| SWARM Pattern | Application | Customer |
|--------------|-------------|----------|
| Protein Swarm | Lead optimization | Pharma R&D |
| ACO | Drug combinations | Oncology |
| PSO | Dose optimization | Clinical pharmacology |
| Hierarchical | Research planning | Research institutes |

---

*QBitaLabs, Inc. — Swarm intelligence for quantum biology and human health*